## MLP model to predict the next character

In [4]:
import torch
import torch.nn.functional as F 
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [6]:
len(words)

32033

In [7]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [68]:
# building the dataset
block_size = 3
X, Y = [], []
for w in words:
    # print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [15]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [ ]:
C = torch.randn((27, 2)) # Embedding 27 characters into lower dimensional space

In [17]:
C

tensor([[-0.0788, -1.3449],
        [ 0.9795,  0.3962],
        [ 0.0653,  1.1604],
        [ 0.0291, -1.4432],
        [ 0.3094, -0.0744],
        [-1.0710,  0.8177],
        [-0.2273, -0.3196],
        [-1.0686,  1.6020],
        [-0.0813,  0.4478],
        [-1.9637, -1.5079],
        [ 0.1681,  0.9619],
        [-0.0705,  0.7990],
        [ 0.8582, -0.0447],
        [ 0.2868,  0.4977],
        [ 0.7427,  2.3305],
        [ 3.4200, -0.1239],
        [ 0.3070, -0.8458],
        [ 0.9399, -1.6255],
        [-0.9308,  0.5018],
        [-1.2755,  0.3028],
        [ 1.6959,  0.7991],
        [-0.2785,  0.1335],
        [-0.2369, -0.5532],
        [ 0.2318, -0.1779],
        [-2.4783,  0.5698],
        [-1.1963,  0.1057],
        [ 0.3191,  1.8595]])

In [19]:
C[5]

tensor([-1.0710,  0.8177])

In [27]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [33]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [ ]:
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1).shape

torch.Size([32, 6])

# Cleaner version

In [69]:
# Parameters
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [70]:
for p in parameters:
    p.requires_grad = True

In [94]:
for _ in range(10000):

    # minibatch construct
    ix = torch.randint(0, X.shape[0], (50,))

    # Forward Pass
    # emb = C[X] 
    emb = C[X[ix]] 
    h = torch.tanh( emb.view(-1, 6) @ W1 + b1 )
    logits = h @ W2 + b2
    # counts = logits.exp()
    # prob = counts / counts.sum(1, keepdim=True)
    # loss = - prob[torch.arange(Y.shape[0]), Y].log().mean()
    # loss = F.cross_entropy(logits, Y)
    loss = F.cross_entropy(logits, Y[ix])
    
    # Backward pass
    for p in parameters:
        p.grad = None

    loss.backward()

    for p in parameters:
        p.data += -0.001 * p.grad

print(loss.item())

2.526524305343628


In [95]:
emb = C[X] 
h = torch.tanh( emb.view(-1, 6) @ W1 + b1 )
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Y)
print('Loss for entire training set: ', loss.item())

Loss for entire training set:  2.3233160972595215
